# SmartBeds - Preprocesado de Datos

## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [1]:
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
import matplotlib.pyplot as plt
from transformers import *

In [2]:
import pickle as pk

### Lectura

In [3]:
import loadData as ld

datos = ld.load(r"../data/")
datos = ld.preprocess(datos)
datos.head()

,DateTime,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,target
7,2018-10-01 21:24:13,4.0,21.0,27.0,35.0,14.0,5.0,2.0,0.0,0.0,0.0,1.0,0.0,False
8,2018-10-01 21:24:14,4.0,20.0,26.0,35.0,14.0,5.0,2.0,0.0,0.0,0.0,1.0,0.0,False
9,2018-10-01 21:24:15,4.0,21.0,26.0,36.0,14.0,5.0,2.0,0.0,0.0,0.0,1.0,0.0,False
10,2018-10-01 21:24:16,2.0,20.0,25.0,34.0,13.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,False
11,2018-10-01 21:24:17,4.0,21.0,27.0,35.0,14.0,6.0,2.0,0.0,0.0,1.0,1.0,0.0,False


In [4]:
from sys import getsizeof
print(getsizeof(datos)/(2**30),'GiB')

0.4823642037808895 GiB


### Datos manuales

In [5]:
datos = ld.newSeizure(datos,'2018-11-10 3:30:00',60*20)
#Las siguientes crisis no tienen momeento final, se ha considerado 15 minutos
datos = ld.newSeizure(datos,'2018-09-29 07:10:00',60*15)
datos = ld.newSeizure(datos,'2019-01-9 23:30:00',60*15)
datos = ld.newSeizure(datos,'2019-01-29 6:12:00',60*15)

### Preprocesado

In [6]:
# Ordención
datos = datos.sort_values(by='DateTime')
with open('data/datos_raw.pdd','wb') as f:
    pk.dump(datos,f)

In [7]:
dateTime=datos['DateTime']
target=datos['target']
only_datos = datos[datos.columns[1:len(datos.columns)-1]]
only_datos.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12
5615198,1.0,14.0,11.0,18.0,2.0,9.0,1.0,0.0,0.0,0.0,1.0,0.0
5615265,1.0,14.0,11.0,18.0,2.0,9.0,1.0,0.0,0.0,0.0,1.0,0.0
5615291,1.0,14.0,11.0,19.0,3.0,9.0,1.0,0.0,0.0,0.0,1.0,0.0
5615346,1.0,14.0,11.0,19.0,3.0,10.0,1.0,0.0,0.0,0.0,1.0,0.0
5615404,1.0,14.0,11.0,18.0,3.0,9.0,1.0,0.0,0.0,0.0,1.0,0.0


In [12]:
#Transformadores
nf = NoiseFilter(minimum=5.0)
norm = Normalizer(max_=100)
bt = ButterTransformer(N=3,Wn=0.05)
vt = VarianceThresholdPD(threshold=0.5)
clean_data = PipelineTransformer(nf,norm).fit_transform(only_datos)
preprocess = PipelineTransformer(bt,vt).fit_transform(clean_data)
clean_data = vt.fit_transform(clean_data)

In [13]:
clean_data.head()

,P1,P2,P3,P4,P5,P6
5615198,0.0,11.47541,9.016393,14.754098,0.0,7.377049
5615265,0.0,11.47541,9.016393,14.754098,0.0,7.377049
5615291,0.0,11.47541,9.016393,15.573770,0.0,7.377049
5615346,0.0,11.47541,9.016393,15.573770,0.0,8.196721
5615404,0.0,11.47541,9.016393,14.754098,0.0,7.377049


In [14]:
preprocess.head()

,P1 BUTTER,P2 BUTTER,P3 BUTTER,P4 BUTTER,P5 BUTTER,P6 BUTTER
5615198,-5.700178e-12,11.475411,9.016399,14.739310,-1.796949e-11,7.365066
5615265,-6.239481e-12,11.475411,9.016399,14.797474,-1.878728e-11,7.379628
5615291,-6.703635e-12,11.475411,9.016400,14.853589,-1.924200e-11,7.394337
5615346,-7.067487e-12,11.475411,9.016401,14.907073,-1.925988e-11,7.408931
5615404,-7.304296e-12,11.475411,9.016401,14.957425,-1.876631e-11,7.423176


In [17]:
datosLimpios = pd.concat((dateTime,clean_data,target),axis=1)
datosProcesados = pd.concat((dateTime,preprocess,target),axis=1)
with open('data/datos_clean.pdd','wb') as f:
    pk.dump(datosLimpios,f)
with open('data/datos_preprocess.pdd','wb') as f:
    pk.dump(datosProcesados,f)

### Cálculo de las  estadísticas móviles
#### Widgets

In [18]:
import gc
gc.collect()

40

In [19]:
import ipywidgets as widgets
from IPython.display import display

slider = widgets.IntSlider(
    value=25,
    min=1,
    max=50,
    description='Ventana:',
    readout=True,
    readout_format='d'
)

display(slider)

IntSlider(value=25, description='Ventana:', max=50, min=1)

In [20]:
window = slider.value
#Transformadores estadisticos
mean = PipelineTransformer(StatisticsTransformer(mode='mean',window=window),norm)
std = PipelineTransformer(StatisticsTransformer(mode='std',window=window),norm)
ran = PipelineTransformer(StatisticsTransformer(mode='range',window=window),norm)
cnt = ConcatenateTransformer(mean,std,ran)

In [ ]:
preprocess_stats = cnt.fit_transform(preprocess)
clean_stats = cnt.fit_transform(clean_data)

In [ ]:
preprocess_stats.head()

In [ ]:
clean_stats.head()

### Exportación

In [18]:
with open('data/datos.pdd','wb') as f:
    pk.dump(datos,f)
    
with open('data/datosE.pdd','wb') as f:
    pk.dump(dataE,f)